In [ ]:
from pyserini.search.lucene import LuceneSearcher
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from tqdm import tqdm
from utils import get_data, creat_index,generate_reformulations,eval_reformulations,make_conversation

data_set="scifact"
data_path= "data/text_data/"
corpus, queries, qrels  =get_data(data_set,data_path=data_path)
queries_ids=list(queries.keys())
document_ids= list(corpus.keys())




  0%|          | 0/5183 [00:00<?, ?it/s]

In [ ]:
index_path="data/index_data/"+data_set+"_docIndex"
index_path=index_path+'/doc_index'     
creat_index(index_path,corpus)
searcher = LuceneSearcher(index_path)
searcher.set_bm25(k1=0.9, b=0.4)

mai 13, 2025 1:12:18 PM org.apache.lucene.store.MMapDirectory lookupProvider


In [7]:

SYSTEM_PROMPT = (
    "A conversation between a User and an Assistant. The User provides a query to be reformulated and rich, "
    "and the Assistant must rewrite the query without asking a question so that a keywords retrieval system can better find the correct document. "
    "No additional text is needed — only the new, reformulated query should be returned.\n"
    "Format: USER: [ORIGINAL QUERY]  ASSISTANT: [NEW QUERY]"
)
SYSTEM_PROMPT2=("Reformulate the query so keywords retrieval system can better find the correct document.\n\n\n [ORIGINAL QUERY]: ")

In [5]:
device="mps"

#Only GRPO
model_name="checkpoints/Qwen_GRPO_clean12/checkpoint-925"
trained_model = AutoModelForCausalLM.from_pretrained(
   model_name,
    torch_dtype="auto",
    device_map=device,
)
trained_tokenizer = AutoTokenizer.from_pretrained(model_name)

#Base model
model_name = "Qwen/Qwen2.5-1.5B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
base_model = AutoModelForCausalLM.from_pretrained(
  model_name,torch_dtype="auto",#, torch_dtype=torch.float16
).to(device)

In [8]:
# For  GRPO-trained model:
trained_outputs = generate_reformulations(
    queries=queries,
    model=trained_model,
    tokenizer=trained_tokenizer,
    device="mps",
    make_conversation=make_conversation,
    system_prompt=SYSTEM_PROMPT,
    offcial_prompt=True
)
# For  original base model:
base_outputs = generate_reformulations(
    queries=queries,
    model=base_model,
    tokenizer=tokenizer,
    device="mps",
    make_conversation=make_conversation,
    system_prompt=SYSTEM_PROMPT,
    offcial_prompt=True
)


Generating: 100%|██████████| 300/300 [04:00<00:00,  1.25it/s]


In [ ]:
eval_reformulations(trained_outputs, queries_ids,qrels,searcher=searcher)

In [ ]:
eval_reformulations(base_outputs, queries_ids,qrels,searcher=searcher)

In [16]:

eval_reformulations(queries.values(),queries_ids,qrels,searcher=searcher)

({'NDCG@1': 0.55333, 'NDCG@10': 0.67891, 'NDCG@100': 0.70669},
 {'Recall@1': 0.53706, 'Recall@10': 0.80383, 'Recall@100': 0.92533},
 {'P@1': 0.55333, 'P@10': 0.08833, 'P@100': 0.01047})